In [10]:
# load packages
import pandas as pd
from easynmt import EasyNMT
import numpy as np
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

### Load and prepare data

In [2]:
# load data
raw = pd.read_parquet('translated.parquet.gzip')

In [27]:
# access your multilingual data and ensure it is lowercase
multi = pd.DataFrame()
multi['trans_text'] = raw.trans_text.str.lower()

### Translate multilingual tickets into English for processing

In [54]:
# build translation model
trans = EasyNMT('mbart50_m2en')

1.47kB [00:00, 94.3kB/s]                                                                                               


C:\Users\keato\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\keato\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [55]:
# translate multilingual to english
strings = multi.trans_text.tolist()

multi['en_text'] = trans.translate(strings, target_lang='en', max_new_tokens=200)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


### Create initial urgency ratings based on sentiment and strength of the negative sentiment

In [120]:
# run sentiment analysis to find negatives and potential negatives
sia = SentimentIntensityAnalyzer()

def v_polarity(text):
    return sia.polarity_scores(text)['compound']

def v_negativity(text):
    return sia.polarity_scores(text)['neg']

multi['polarity'] = multi['en_text'].apply(v_polarity)
multi['negativity'] = multi['en_text'].apply(v_negativity)

In [313]:
# label tickets as positive, negative, or neutral
min_positive = 0.3
min_neutral = 0

multi['polarity_class'] = np.select([((multi['polarity'] > min_positive) & ((multi['negativity'] == 0) | (multi['polarity'] > 0.7))), 
                                     ((multi['polarity'] < min_neutral) | (multi['negativity'] > 0)), 
                                     ((min_positive >= multi['polarity']) & (multi['polarity'] >= min_neutral))],
                                    ['Positive', 'Negative', 'Neutral'])

In [348]:
# classify urgency based on level of negativity
high_urgency_max = -0.5
mid_urgency_max = -0.35
low_urgency_min = -0.2

multi['urgency_polarity'] = np.select([(multi['polarity'] < high_urgency_max),
                                       ((multi['polarity'] > low_urgency_min) & (multi['polarity_class'] != "Positive")),
                                       ((mid_urgency_max <= multi['polarity']) & (multi['polarity'] <= low_urgency_min)),
                                       ((high_urgency_max <= multi['polarity']) & (multi['polarity'] <= mid_urgency_max))],
                                     [4, 1, 2, 3],
                                     -5).astype(int)

### Adjust urgency rating based on words and characters used in the ticket

In [349]:
# keywords for urgency; score based on number of words in text
urgent_words = ['need', 'urgent', 'urgency', 'urgently', 'please', 'help', 'useless', 'immediate', 'immediately', 'dire', 'asap', 'pay', 'paid', 'worst', 'worse', 'terrible', 'terribly', 'broke', 'broken', 'disappoint', 'disappointed', 'disappointingly', 'disappointing', 'quick', 'fast']
urgent_symbols = ['!', '$', '?']

def urgent_word_count(text):
    count = 0
    for symbol in urgent_symbols:
        count += text.count(symbol)/2
    
    text = ''.join([a for a in text if a.isalpha() or a == " "])
    
    to_check = text.split()
    for word in urgent_words:
        count += to_check.count(word)
  
    return count

In [350]:
# get counts of urgent text
multi['urgency_text'] = multi['en_text'].apply(urgent_word_count)

In [352]:
# get key urgency stats
urgency_text_90th_p = multi.urgency_text.iloc[round(len(multi.urgency_text)*.1)]
urgency_text_75th_p = multi.urgency_text.iloc[round(len(multi.urgency_text)*.25)]

In [353]:
# increase urgency level based on num of urgency text score
multi['urgency_rating'] = np.where(((multi['polarity'] != 'Positive') & (multi['urgency_text'] > urgency_text_75th_p) & (multi['urgency_polarity'] > 0)),
                                  multi['urgency_polarity'] + 1,
                                  multi['urgency_polarity'])

multi['urgency_rating'] = np.where(((multi['polarity'] != 'Positive') & (multi['urgency_text'] > urgency_text_90th_p) & (multi['urgency_polarity'] > 0)),
                                  multi['urgency_rating'] + 1,
                                  multi['urgency_rating'])

### Create final urgency classification based on above

In [365]:
# create classification based on urgency ratings
multi['urgency_class'] = np.select([(multi['urgency_rating'] == 6), (multi['urgency_rating'] == 5), (multi['urgency_rating'] == 4), (multi['urgency_rating'] == 3), (multi['urgency_rating'] == 2), ((multi['urgency_rating'] == 1) & (multi['urgency_text'] > 0)), ((multi['urgency_rating'] == 1) & (multi['urgency_text'] == 0))],
                                  ['HIGH', 'HIGH', 'Mid-High', 'Low-Mid', 'Low', 'Lowest', 'None'],
                                  'None')

In [370]:
# make urgency classes sortable
multi.urgency_class = pd.Categorical(multi.urgency_class,
                                    categories=['None', 'Lowest', 'Low', 'Low-Mid', 'Mid-High', 'HIGH'],
                                    ordered=True)

multi.sort_values(by='urgency_class', ascending=False)